### Laboratory Task 4

Instruction: Train a linear regression model in PyTorch using a regression dataset. Use the following parameters.

Criterion: MSE Loss, Fully Connected Layers x 2, Batch Size: 8, Optimizer: SGD, Epoch: 1000

In [1]:
import torch
import numpy as np
import sys
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn as nn

In [2]:
torch.__version__

'2.8.0+cpu'

In [3]:
#We can check whether we have gpu
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
print("Device: ", device)

Device:  cpu


In [4]:
# Input (temp, rainfall, humidity)
x_train = np.array([
    [73, 67, 43], [91, 88, 64], [87, 134, 58], 
    [102, 43, 37], [69, 96, 70], [73, 67, 43], 
    [91, 88, 64], [87, 134, 58], [102, 43, 37], 
    [69, 96, 70], [73, 67, 43], [91, 88, 64], 
    [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                   dtype='float32')

# Targets (apples, oranges)
y_train = np.array([
    [56, 70], [81, 101], [119, 133], 
    [22, 37], [103, 119], [56, 70], 
    [81, 101], [119, 133], [22, 37], 
    [103, 119], [56, 70], [81, 101], 
    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

In [5]:
inputs = torch.from_numpy(x_train)
targets = torch.from_numpy(y_train)
print(inputs.size())
print(targets.size())

torch.Size([15, 3])
torch.Size([15, 2])


In [6]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

## Batch Size (8)

In [7]:
# Define data loader
batch_size = 8
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [8]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 73.,  67.,  43.],
        [ 69.,  96.,  70.],
        [102.,  43.,  37.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 91.,  88.,  64.],
        [ 91.,  88.,  64.],
        [ 73.,  67.,  43.]])
tensor([[ 56.,  70.],
        [103., 119.],
        [ 22.,  37.],
        [ 22.,  37.],
        [103., 119.],
        [ 81., 101.],
        [ 81., 101.],
        [ 56.,  70.]])


In [9]:
# Define model
model = nn.Linear(3, 2)  #nn.Linear assume this shape (in_features, out_features)
print(model.weight)
print(model.weight.size()) # (out_features, in_features)
print(model.bias)
print(model.bias.size()) #(out_features)

Parameter containing:
tensor([[-0.2155,  0.5213, -0.4490],
        [-0.0497,  0.5116, -0.3600]], requires_grad=True)
torch.Size([2, 3])
Parameter containing:
tensor([ 0.4815, -0.2034], requires_grad=True)
torch.Size([2])


In [10]:
# Parameters
list(model.parameters())  #model.param returns a generator

[Parameter containing:
 tensor([[-0.2155,  0.5213, -0.4490],
         [-0.0497,  0.5116, -0.3600]], requires_grad=True),
 Parameter containing:
 tensor([ 0.4815, -0.2034], requires_grad=True)]

In [11]:
#we can print the complexity by the number of parameters
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

8


In [12]:
# Generate predictions
preds = model(inputs)
preds

tensor([[  0.3673,  14.9628],
        [ -1.9948,  17.2507],
        [ 25.5392,  43.1428],
        [-15.6971,   3.4035],
        [  4.2213,  20.2768],
        [  0.3673,  14.9628],
        [ -1.9948,  17.2507],
        [ 25.5392,  43.1428],
        [-15.6971,   3.4035],
        [  4.2213,  20.2768],
        [  0.3673,  14.9628],
        [ -1.9948,  17.2507],
        [ 25.5392,  43.1428],
        [-15.6971,   3.4035],
        [  4.2213,  20.2768]], grad_fn=<AddmmBackward0>)

## MSE Loss

In [13]:
criterion_mse = nn.MSELoss()
criterion_softmax_cross_entropy_loss = nn.CrossEntropyLoss()

In [14]:
mse = criterion_mse(preds, targets)
print(mse)
print(mse.item())  ##print out the loss number

tensor(5888.8706, grad_fn=<MseLossBackward0>)
5888.87060546875


## Optimizer (SGD)

In [15]:
# Define optimizer
#momentum update the weight based on past gradients also, which will be useful for getting out of local max/min
#If our momentum parameter was $0.9$, we would get our current grad + the multiplication of the gradient 
#from one time step ago by $0.9$, the one from two time steps ago by $0.9^2 = 0.81$, etc.

opt = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

In [16]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            xb.to(device) #move them to gpu if possible, if not, it will be cpu
            yb.to(device)
                    
            # 1. Predict
            pred = model(xb)
                      
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Calculate gradient
            opt.zero_grad()  #if not, the gradients will accumulate
            loss.backward()
            
            # Print out the gradients.
            # print ('dL/dw: ', model.weight.grad) 
            # print ('dL/db: ', model.bias.grad)
            
            # 4. Update parameters using gradients
            opt.step()
            
        # Print the progress
        if (epoch+1) % 10 == 0:
            sys.stdout.write("\rEpoch [{}/{}], Loss: {:.4f}".format(epoch+1, num_epochs, loss.item()))

## Epoch (1000)

In [17]:
#train for 1000 epochs
fit(1000, model, criterion_mse, opt, train_dl)

Epoch [90/1000], Loss: 0.8770

Epoch [190/1000], Loss: 2.6717

Epoch [290/1000], Loss: 7.1962

Epoch [380/1000], Loss: 0.6992

Epoch [490/1000], Loss: 5.2523

...

Epoch [680/1000], Loss: 0.7624

Epoch [780/1000], Loss: 0.6888

Epoch [880/1000], Loss: 3.2325

Epoch [970/1000], Loss: 0.7085

Epoch [1000/1000], Loss: 0.6590

In [18]:
# Generate predictions
preds = model(inputs)
loss = criterion_mse(preds, targets)
print(loss.item())

0.5599416494369507
